# CNN

## Notebook's Environment

In [ ]:
INSTALL_DEPS = True
if INSTALL_DEPS:
  %pip install hurst==0.0.5
  %pip install imbalanced_learn==0.12.3
  %pip install imblearn==0.0
  %pip install protobuf==5.27.0
  %pip install pykalman==0.9.7
  %pip install tqdm==4.66.4

!python --version

## Cloud Environment Setup

In [ ]:
import os
import sys
import warnings

warnings.filterwarnings("ignore")

IN_KAGGLE = IN_COLAB = False
try:
    # https://www.tensorflow.org/install/pip#windows-wsl2
    import google.colab
    from google.colab import drive

    drive.mount("/content/drive")
    DATA_PATH = "/content/drive/MyDrive/EDT dataset"
    MODEL_PATH = "/content/drive/MyDrive/models"
    IN_COLAB = True
    print("Colab!")
except:
    IN_COLAB = False
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ and not IN_COLAB:
    print("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    MODEL_PATH = "./models"
    DATA_PATH = "/kaggle/input/intra-day-agriculture-futures-trades-2023-2024"
    IN_KAGGLE = True
    print("Kaggle!")
elif not IN_COLAB:
    IN_KAGGLE = False
    MODEL_PATH = "./models"
    DATA_PATH = "./data/"
    print("running localhost!")

In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

print(f'Tensorflow version: [{tf.__version__}]')

tf.get_logger().setLevel('INFO')

#tf.config.set_soft_device_placement(True)
#tf.config.experimental.enable_op_determinism()
#tf.random.set_seed(1)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
except Exception as e:
  gpus = tf.config.experimental.list_physical_devices('GPU')
  if len(gpus) > 0:
    try:
        strategy = tf.distribute.MirroredStrategy()
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    finally:
        print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
  else:
    # CPU is final fallback
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

def is_tpu_strategy(strategy):
    return isinstance(strategy, tf.distribute.TPUStrategy)

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

# Instruments

## Data Load

In [ ]:
import pandas as pd
import numpy as np
from algo_trading_utility_script import *

filename = f"{DATA_PATH}{os.sep}futures_{INTERVAL}.csv"
print(filename)
futs_df = pd.read_csv(filename, index_col="Date", parse_dates=True)

print(futs_df.shape)

HALF_LIFE, HURST = get_ou(futs_df, f'{TARGET_FUT}_Close')

print("Half-Life:", HALF_LIFE)
print("Hurst:", HURST)

futs_df.head(2)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.plot(futs_df[f'{TARGET_FUT}_Close'], label=f'{TARGET_FUT} Close', alpha=0.7)
plt.title(f'{TARGET_FUT} Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Prepare the Data

In [ ]:
import pickle
from tqdm import tqdm

from sklearn.preprocessing import normalize

TEST_SPLIT = 0.8
TRAIN_SIZE = int(len(futs_df) * TEST_SPLIT)
CACHE = True
FUTURES_TMP_FILE = "./tmp/futures.pkl"
os.makedirs("./tmp/", exist_ok=True)

def oversample_mean_reversions(train_agri_ts, window, period=INTERVAL, hurst=HURST):
    samples = []
    for df in tqdm(train_agri_ts, desc="oversample_mean_reversions"):
        bb_df = df.copy()
        results_df = param_search_bbs(bb_df, StockFeatExt.CLOSE, period, initial_window=window * 2, window_min=window // 2, hurst=hurst)
        results_df = results_df[results_df["Metric"] == "Sharpe"]
        bb_df, _ = bollinger_band_backtest(bb_df, StockFeatExt.CLOSE, results_df["Window"].iloc[0], period, std_factor=results_df["Standard_Factor"].iloc[0])

        samples.append(bb_df[train_agri_ts[0].columns].reset_index(drop=True))
    return train_agri_ts + samples

def normalize_and_label_data(ts, meta_label=META_LABEL, cols_to_scale=COLS_TO_SCALE):
    y0 = 0
    y1 = 0
    process_data = []
    for df in tqdm(ts, desc="label_data"):
        df = aug_metalabel_mr(df)
        y0  += (df[meta_label] == 0).sum()
        y1  += (df[meta_label] > 0).sum()

        if cols_to_scale is not None:
            aug_df_scaled = normalize(df[cols_to_scale])
            aug_df_scaled = pd.DataFrame(aug_df_scaled, columns=cols_to_scale)
            df[cols_to_scale] = aug_df_scaled
            df = df.loc[:, ~df.columns.duplicated(keep="first")]

        process_data.append(df)

    total = y0 + y1
    class_weight_0 = total / y0 if y0 != 0 else 0
    class_weight_1 = total / y1 if y1 != 0 else 0
    class_weights = {0: class_weight_0, 1: class_weight_1}

    return process_data, class_weights

with strategy.scope():
    if not os.path.exists(FUTURES_TMP_FILE):
        futs_exog_df = process_exog(MARKET_FUTS, futs_df)
        train_agri_ts, val_agri_ts = process_futures(FUTS, futs_df, futs_exog_df, TRAIN_SIZE, INTERVAL)
        train_agri_ts = oversample_mean_reversions(train_agri_ts, HALF_LIFE)
        val_agri_ts = oversample_mean_reversions(val_agri_ts, HALF_LIFE)

        if CACHE:
            with open(FUTURES_TMP_FILE, 'wb') as f:
                pickle.dump((train_agri_ts, val_agri_ts), f)
    else:
        with open(FUTURES_TMP_FILE, 'rb') as f:
            train_agri_ts, val_agri_ts = pickle.load(f)

train_agri_ts, CLASS_WEIGHTS = normalize_and_label_data(train_agri_ts)
val_agri_ts, val_weights = normalize_and_label_data(val_agri_ts)

print(f"train: {CLASS_WEIGHTS}")
print(f"test: {val_weights}")
np.shape(train_agri_ts)

In [ ]:
FEATURES = KF_COLS + BB_COLS + MOM_COLS + StockFeatExt.list

sample = val_agri_ts[0]
print(sample[META_LABEL].value_counts())

sampled_pattenrs = sample[sample[META_LABEL] > 0]
sampled_pattenrs[FEATURES + [META_LABEL, "Ret"]].tail(10)

In [ ]:
WINDOW = 8
WINDOW_TMP_PATH = "./tmp/"
# TPU see: https://github.com/tensorflow/tensorflow/issues/41635
BATCH_SIZE = 8  * strategy.num_replicas_in_sync # Default 8
print(f"BATCH_SIZE: {BATCH_SIZE}")

def prepare_windows(data_df, label_df, window_size=WINDOW):
    """
    Prepare windows of features and corresponding labels for classification.

    Parameters:
    - data_df: DataFrame containing the features.
    - label_df: DataFrame containing the labels.
    - window_size: The size of the input window.

    Returns:
    - X: Array of input windows.
    - y: Array of corresponding labels.
    """
    X, y = [], []
    for i in range(len(data_df) - window_size):
        input_window = data_df.iloc[i : i + window_size].values
        assert not np.isnan(input_window).any(), "NaN values found in input window"
        X.append(input_window)
        if label_df is not None:
            target_label = label_df.iloc[i + window_size]
            y.append([target_label])
            assert not np.isnan(target_label).any(), "NaN values found in target label"
    return np.array(X), np.array(y)

def prepare_windows_with_disjoint_ts(ts_list, window_size=WINDOW):
    """
    Generator function to yield windows of features and corresponding labels from multiple time series.

    Parameters:
    - ts_list: List of DataFrames, each containing a time series.
    - window_size: The size of the input window.

    Yields:
    - features: The input window of features.
    - labels: The corresponding label.
    """
    for data_df in ts_list:
        X, y = prepare_windows(data_df[FEATURES], data_df[META_LABEL], window_size=window_size)
        for features, labels in zip(X, y):
            yield features, labels

def create_tf_dataset_from_generator(ts_list, window_size=WINDOW, batch_size=BATCH_SIZE):
    """
    Create a TensorFlow dataset from a generator.

    Parameters:
    - ts_list: List of DataFrames, each containing a time series.
    - window_size: The size of the input window.
    - batch_size: The batch size for the dataset.

    Returns:
    - dataset: A TensorFlow dataset.
    """
    dataset = tf.data.Dataset.from_generator(
        lambda: prepare_windows_with_disjoint_ts(ts_list, window_size=window_size),
        output_signature=(
            tf.TensorSpec(shape=(window_size, len(FEATURES)), dtype=tf.float32),
            tf.TensorSpec(shape=(1,), dtype=tf.float32)  # Assuming labels are floats for binary classification
        )
    )
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

with strategy.scope():
    train_dataset = create_tf_dataset_from_generator(train_agri_ts)
    val_dataset = create_tf_dataset_from_generator(val_agri_ts)

# CNN Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Add, Multiply, Input, Flatten, Dense, AveragePooling1D, SpatialDropout1D, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

MODEL_NAME = "WAVENET"
MAX_DILATION = 8
DILATION_RATE = 1
EPOCHS = 30
PATIENCE_EPOCHS = 8
FILTERS = 10
DROPRATE = 0.4
KERNEL_SIZE = 4
REG_WEIGHTS = 0.01
CONVOLUTIONS = 8
BATCH_SIZE = 10
LEARN_RATTE =1e-4

def wavenet_block(inputs, layer_id, filters, kernel_size=KERNEL_SIZE, 
                  dilation_rate=DILATION_RATE, reg_param=REG_WEIGHTS, dropout_rate=DROPRATE):
    conv_f = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal',
                    kernel_regularizer=l2(reg_param), name=f'conv_f_{layer_id}')(inputs)
    conv_g = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal',
                    kernel_regularizer=l2(reg_param), name=f'conv_g_{layer_id}')(inputs)
    tanh_out = Activation('tanh')(conv_f)
    sigmoid_out = Activation('sigmoid')(conv_g)
    merged = Multiply()([tanh_out, sigmoid_out])
    merged = SpatialDropout1D(dropout_rate)(merged)

    skip_out = Conv1D(filters, 1, padding='same', kernel_regularizer=l2(reg_param), name=f'skip_{layer_id}')(merged)
    residual_out = Conv1D(filters, 1, padding='same', kernel_regularizer=l2(reg_param), name=f'residual_{layer_id}')(inputs)
    residual_out = Add()([residual_out, skip_out])

    return residual_out, skip_out


def build_wavenet_model(input_shape, filters=FILTERS, kernel_size=KERNEL_SIZE, 
                        reg_param=REG_WEIGHTS, dropout_rate=DROPRATE, convolutions=CONVOLUTIONS, 
                        max_dilation=MAX_DILATION):
    inputs = Input(shape=input_shape)
    x = inputs
    skip_connections = []

    for i in range(convolutions):
        dilation_rate = min(2 ** i, max_dilation)
        x, skip = wavenet_block(x, layer_id=i,filters=filters, 
                                kernel_size=kernel_size, dilation_rate=dilation_rate,  
                                reg_param=reg_param, dropout_rate=dropout_rate)
        skip_connections.append(skip)

    x = Add()(skip_connections)
    x = Activation('relu')(x)

    # Downsample the output to coarser frames
    x = AveragePooling1D(pool_size=2)(x)

    x = Conv1D(filters, 1, activation='relu', kernel_regularizer=l2(reg_param), name='post_conv_1')(x)
    x = Conv1D(filters, 1, activation='relu', kernel_regularizer=l2(reg_param), name='post_conv_2')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(reg_param), name='fc1')(x)
    x = Dropout(dropout_rate, name='fcdo1')(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(reg_param), name='fc1')(x)
    x = Dropout(dropout_rate, name='fcdo1')(x)
    outputs = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param), name='output_dense')(x)

    model = Model(inputs, outputs, name=MODEL_NAME)
    return model

## Training

In [ ]:
import tensorflow as tf

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

MODEL_DIR = f"models/{MODEL_NAME}"
IMAGES_DIR = f"images/{MODEL_NAME}/images"
LOG_BASEPATH = f"logs/{MODEL_NAME}/tb"

def build_cnn(input_shape, train_dataset, test_dataset=None, lr=LEARN_RATTE):
    model = build_wavenet_model(
        input_shape=input_shape,
        filters=FILTERS,
        kernel_size=KERNEL_SIZE,
        reg_param=REG_WEIGHTS,
        dropout_rate=DROPRATE,
        convolutions=CONVOLUTIONS,
    )

    optimizer = Adam(learning_rate=lr)
    model.compile(
        loss=BinaryCrossentropy(),
        optimizer=optimizer,
        metrics=[BinaryAccuracy(name='binary_accuracy'), AUC(name='auc')]
    )
    callbacks = [
        EarlyStopping(
            patience=PATIENCE_EPOCHS,
            monitor="val_loss",
            restore_best_weights=True,
        ),
        ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=2,
            verbose=1,
            min_lr=1e-5
        ),
        TensorBoard(
            log_dir=LOG_BASEPATH,
            histogram_freq=1,
            write_images=True
        )
    ]

    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    return model, history

with strategy.scope():
    input_shape = (WINDOW, len(FEATURES))
    print(f"input_shape: {input_shape}")

    model, history = build_cnn(input_shape, train_dataset=train_dataset, test_dataset=val_dataset)
    model.save(MODEL_PATH)
    model.summary()

## Metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, roc_auc_score

def print_metrics():
    ypred_raw = model.predict([X_t])
    pred = (ypred_raw > 0.5).astype(int)
    metrics = {}

    metrics = {
        "Accuracy": accuracy_score(y_t.flatten(), pred.flatten()),
        "Precision": precision_score(y_t.flatten(), pred.flatten()),
        "Recall": recall_score(y_t.flatten(), pred.flatten()),
        "F1b Score": fbeta_score(y_t.flatten(), pred.flatten(), average="weighted", beta=0.1),
        "ROC AUC": roc_auc_score(y_t.flatten(), ypred_raw.flatten(), average='weighted')  # Using raw probabilities
    }

    metrics_unseen_df = pd.DataFrame.from_dict(metrics, orient='index')
    metrics_unseen_df

In [ ]:
from tensorflow.math import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(cm, labels, cm2=None, labels2=None):
        plt.figure(figsize=(8 if cm2 is not None else 4, 4))
        if cm2 is not None:
            plt.subplot(1, 2, 1)
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)

        df_cm = pd.DataFrame((cm / np.sum(cm, axis=1)[:, None])*100, index=[i for i in labels], columns=[i for i in labels])
        cm_plot1 = sns.heatmap(df_cm, annot=True,  fmt=".2f", cmap='Blues', xticklabels=labels, yticklabels=labels).get_figure()
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.title('Confusion Matrix 1')
        tick_marks = np.arange(len(labels))
        plt.xticks(tick_marks, labels, rotation=45)
        plt.yticks(tick_marks, labels)
        plt.tight_layout()

        return cm_plot1

cm = confusion_matrix(y_t.flatten(), pred)
figure = plot_confusion_matrix(cm, labels=[1,0])

# Grid Search

In [ ]:
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit
from tensorboard.plugins.hparams import api as hp
from tensorflow.summary import create_file_writer
import json

HP_KERNEL_SIZE = hp.HParam("kernel_size", hp.Discrete([KERNEL_SIZE * 2, KERNEL_SIZE]))
HP_BATCH_SIZE = hp.HParam("batch_size", hp.Discrete([BATCH_SIZE]))
HP_EPOCHS = hp.HParam("epochs", hp.Discrete([EPOCHS]))
HP_DILATION_RATE = hp.HParam("dilation_rate", hp.Discrete([DILATION_RATE]))
HP_DROPOUT_RATE = hp.HParam("dropout_rate", hp.Discrete([DROPRATE, DROPRATE*2]))
HP_REG_WEIGHTS = hp.HParam("reg_weight", hp.Discrete([REG_WEIGHTS, REG_WEIGHTS*2]))
HP_LEARNING_RATE = hp.HParam("learning_rate", hp.Discrete([LEARN_RATE]))
HP_PATIENCE = hp.HParam("patience", hp.Discrete([PATIENCE_EPOCHS]))
HP_ALPHA = hp.HParam("alpha", hp.Discrete([ERROR_ALPHA, ERROR_ALPHA-0.5, ERROR_ALPHA+0.5]))
HP_GAMMA = hp.HParam("gamma", hp.Discrete([ERROR_GAMMA, ERROR_GAMMA-0.5, ERROR_GAMMA+0.5]))
HP_HIDDEN_DENSE = hp.HParam("dense_units", hp.Discrete([
    f"{WINDOW}",
    f"{WINDOW*2}_{WINDOW}",
    f"{WINDOW*2}_{WINDOW}_{WINDOW//2}",
    f"{WINDOW*4}_{WINDOW*2}",
]))
HP_FILTERS = hp.HParam("filters", hp.Discrete([FILTERS //2 ,FILTERS, FILTERS * 2]))
HPARAMS = [
    HP_FILTERS,
    HP_KERNEL_SIZE,
    HP_BATCH_SIZE,
    HP_EPOCHS,
    HP_DILATION_RATE,
    HP_DROPOUT_RATE,
    HP_REG_WEIGHTS,
    HP_LEARNING_RATE,
    HP_PATIENCE,
    HP_HIDDEN_DENSE,
    HP_ALPHA,
        HP_GAMMA
    ]

def grid_search_build_cnn(input_shape, X, y, Xt=None, yt=None, hparams=HPARAMS, file_name=f"best_params.json", checkpoint_file = f"checkpoint.json"):
    def _decode_arrays(config_str):
        return [int(unit) for unit in config_str.split('_')]

    def _save_best_params(best_params, best_loss, best_metric, other_metrics = None, file_name="best_params.json"):
        os.makedirs(MODEL_DIR, exist_ok=True)
        with open(f"{MODEL_DIR}/{file_name}", "w") as file:
            json.dump({"best_params": best_params, "best_loss": best_loss, "best_metric": best_metric, 'other_metrics': other_metrics}, file)

    def _load_checkpoint(file_name):
        json = None
        try:
            os.makedirs(MODEL_DIR, exist_ok=True)
            with open(f"{MODEL_DIR}/{file_name}", "r") as file:
                json = json.load(file)
        except Exception as e:
            print(f"File {MODEL_DIR}/{file_name} not found or error {e}")
        return json

    def _save_checkpoint(state, file_name):
        os.makedirs(MODEL_DIR, exist_ok=True)
        with open(f"{MODEL_DIR}/{file_name}", "w") as file:
            json.dump(state, file)

    with create_file_writer(f"{LOG_BASEPATH}/hparam_tuning").as_default():
        hp.hparams_config(
            hparams=hparams,
            metrics=[hp.Metric(TARGET_METRIC, display_name=TARGET_METRIC)],
        )

    start_index = 0
    best_loss = np.inf
    best_metric = -np.inf
    best_params = None
    checkpoint = _load_checkpoint(checkpoint_file)
    if checkpoint:
        start_index = checkpoint['next_index']
        best_loss = checkpoint['best_loss']
        best_metric = checkpoint['best_metric']
        best_params = checkpoint['best_params']

    grid = list(ParameterGrid({h.name: h.domain.values for h in hparams}))
    for index, hp_values in enumerate(tqdm(grid[start_index:], desc="Grid Search.."), start=start_index):
        dense_units = _decode_arrays(hp_values["dense_units"])
        filters = _decode_arrays(hp_values["filters"])
        b = hp_values["bias"]
        k = hp_values["kernel_size"]
        d = hp_values["dilation_rate"]
        rw = hp_values["reg_weight"]
        drop = hp_values["dropout_rate"]

        ERROR_ALPHA = hp_values["alpha"]
        ERROR_GAMMA = hp_values["gamma"]
        print(f"Shapes{input_shape}: x{X[0].shape}xg{X[1].shape}y{y.shape}, filters {filters}, dense {dense_units}, k: {k}, d: {d}, rw: {rw}, drop: {drop}, b: {b}, alpha: {ERROR_ALPHA},  gamma: {ERROR_GAMMA}")

        model, history = build_cnn(input_shape, X, y,
                                    Xt=Xt, yt=yt,
                                    filters=filters,
                                    kernel_size=k,
                                    b_cv=True)
        loss = history.history[f"val_loss"][-1]
        metric = history.history[f"val_{TARGET_METRIC}"][-1]
        if (metric > best_metric):
            best_history = history
            best_loss = loss
            best_metric = metric
            best_model = model
            best_params = hp_values
            other_metrics = {
                f"{TARGET_METRIC}": history.history[f"{TARGET_METRIC}"][-1],
                f"v_{TARGET_METRIC}": history.history[f"val_{TARGET_METRIC}"][-1],
                'ba': history.history["ba"][-1],
                'v_ba': history.history["val_ba"][-1],
            }
            _save_best_params(best_params, best_loss, best_metric, other_metrics, file_name)
        checkpoint_state = {
            'next_index': index + 1,
            'best_loss': best_loss,
            'best_metric': best_metric,
            'best_params': best_params
        }
        _save_checkpoint(checkpoint_state, checkpoint_file)
    return best_model, best_history, best_params, best_loss, best_metric

PARAM_SEARCH = False
if PARAM_SEARCH:
    with strategy.scope():
        assert not np.any(pd.isna(X)) and not np.any(np.isnan(X_t))
        print(f"{X.shape}")
        input_shape = (
            WINDOW,
            1 if len(X.shape) < 3 else X.shape[2],
        )

        model, history, best_params, best_loss, best_metric = grid_search_build_cnn(input_shape, X=X, y=y, Xt=X_t, yt=y_t, hparams=HPARAMS)
        print(best_params)
        print(best_metric)

# CV

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

CV_MODEL = True
CV_SPLITS = 3

def train_cv_model(X, y, input_shape, n_splits=5, perturb=True, window=WINDOW):
    def _perturb_gaussiannoise(X, noise_level=0.1):
        sigma = noise_level * np.std(X)
        noise = np.random.normal(0, sigma, X.shape)
        return X + noise

    if perturb:
        X = _perturb_gaussiannoise(X)

    results = []
    tscv = TimeSeriesSplit(n_splits=n_splits)
    global metrics_col
    metrics_col = None

    for train_index, test_index in tqdm(tscv.split(X), desc=f"CV Testing for n_splits: {n_splits}"):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]

        X_train_windows, y_train_windows = prepare_windows(X_train, y_train, window_size=window)
        X_test_windows, y_test_windows = prepare_windows(X_test, y_test, window_size=window)

        try:
            cv_model, _ = build_cnn(input_shape, X=X_train_windows, y=y_train_windows, Xt=X_test_windows, yt=y_test_windows)

            result = cv_model.evaluate([X_test_windows], y_test_windows, verbose=0)
            results.append(result)

            if metrics_col is None:
                metrics_col = cv_model.metrics
        except Exception as e:
            print(f"CV error on fold with exception: {e}")

    if metrics_col is None:
        raise ValueError("No successful model training; metrics_col is None")

    metrics_names = [metric.name for metric in metrics_col]
    results_df = pd.DataFrame(results, columns=metrics_names)

    return results_df

# results_df = train_cv_model(train_ts_df.drop(columns=[META_LABEL]), train_ts_df[META_LABEL], input_shape)
# results_df